# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('tic-tac-toe.csv')
data.head()


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [2]:
data.shape

(958, 10)

In [3]:
data.isnull().sum()

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

In [4]:
data.dtypes

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object

In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to each of the feature columns
for col in data.columns[:-1]:  # Exclude the 'class' column
    data[col] = label_encoder.fit_transform(data[col])

# Map the 'class' column to binary values
data['class'] = data['class'].map({True: 1, False: 0})


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [6]:
predictors = data.drop("class", axis=1) # all columns except class
target = data['class'] # class column

In [8]:
predictors.head()


,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,2,2,2,2,1,1,2,1,1
1,2,2,2,2,1,1,1,2,1
2,2,2,2,2,1,1,1,1,2
3,2,2,2,2,1,1,1,0,0
4,2,2,2,2,1,1,0,1,0


In [9]:
n_cols = predictors.shape[1] # number of predictors

In [65]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(150, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(150, activation='relu'))
    model.add(Dense(150, activation='relu'))
    model.add(Dense(150, activation='relu'))
    model.add(Dense(50, activation='relu'))

    model.add(Dense(1, activation='softmax'))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [66]:
from sklearn.model_selection import train_test_split

# build the model
model = regression_model()

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

# fit the model
model.fit(X_train, y_train, epochs=300, verbose=2)

Epoch 1/300


c:\Users\Salva\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


21/21 - 6s - 301ms/step - loss: 0.3537
Epoch 2/300
21/21 - 0s - 7ms/step - loss: 0.3537
Epoch 3/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 4/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 5/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 6/300
21/21 - 0s - 7ms/step - loss: 0.3537
Epoch 7/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 8/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 9/300
21/21 - 0s - 8ms/step - loss: 0.3537
Epoch 10/300
21/21 - 0s - 8ms/step - loss: 0.3537
Epoch 11/300
21/21 - 0s - 9ms/step - loss: 0.3537
Epoch 12/300
21/21 - 0s - 7ms/step - loss: 0.3537
Epoch 13/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 14/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 15/300
21/21 - 0s - 7ms/step - loss: 0.3537
Epoch 16/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 17/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 18/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 19/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 20/300
21/21 - 0s - 6ms/step - loss: 0.3537
Epoch 21/300
21/21 

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [67]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate and print regression metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'R² Score: {r2:.4f}')

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  
Mean Squared Error (MSE): 0.3299
Mean Absolute Error (MAE): 0.3299
R² Score: -0.4922


c:\Users\Salva\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [83]:
from tensorflow.keras.optimizers import Adam

# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(150, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(150, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1))
    adam_optimizer = Adam(learning_rate=0.003)  # adjust the learning rate if needed

    # compile model
    model.compile(optimizer=adam_optimizer, loss='mean_squared_error')
    return model

# build the model
model = regression_model()

# fit the model
model.fit(X_train, y_train, epochs=200, verbose=2)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate and print regression metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'R² Score: {r2:.4f}')

c:\Users\Salva\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
21/21 - 5s - 261ms/step - loss: 0.2480
Epoch 2/200
21/21 - 0s - 6ms/step - loss: 0.2069
Epoch 3/200
21/21 - 0s - 6ms/step - loss: 0.1700
Epoch 4/200
21/21 - 0s - 6ms/step - loss: 0.1368
Epoch 5/200
21/21 - 0s - 6ms/step - loss: 0.1232
Epoch 6/200
21/21 - 0s - 5ms/step - loss: 0.0991
Epoch 7/200
21/21 - 0s - 5ms/step - loss: 0.0856
Epoch 8/200
21/21 - 0s - 5ms/step - loss: 0.0688
Epoch 9/200
21/21 - 0s - 5ms/step - loss: 0.0636
Epoch 10/200
21/21 - 0s - 5ms/step - loss: 0.0554
Epoch 11/200
21/21 - 0s - 6ms/step - loss: 0.0469
Epoch 12/200
21/21 - 0s - 5ms/step - loss: 0.0276
Epoch 13/200
21/21 - 0s - 5ms/step - loss: 0.0162
Epoch 14/200
21/21 - 0s - 5ms/step - loss: 0.0579
Epoch 15/200
21/21 - 0s - 5ms/step - loss: 0.0664
Epoch 16/200
21/21 - 0s - 6ms/step - loss: 0.0524
Epoch 17/200
21/21 - 0s - 6ms/step - loss: 0.0190
Epoch 18/200
21/21 - 0s - 6ms/step - loss: 0.0127
Epoch 19/200
21/21 - 0s - 6ms/step - loss: 0.0126
Epoch 20/200
21/21 - 0s - 6ms/step - loss: 0.0379
Epoch 2

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
#HAving the ouptut neuron as linear is significantly better than Softmax. The increase in neurons and layers gave a good improvement, until more layers made no further improvement. Increase o epoch to around 300 also helped. Playing with learning rate improve things slighlty depending also on the other variables. Leading to diminishing returns at some point. Same as the other parameters)